<a href="https://colab.research.google.com/github/NamrathaKaranth/30-Days-of-Deep-Learning/blob/main/mnist_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd  "/content/gdrixve/Shared drives/Datasets/sridhar-30-days"

[Errno 2] No such file or directory: '/content/gdrixve/Shared drives/Datasets/sridhar-30-days'
/content


In [ ]:
!mkdir gan_mnist

In [ ]:
%cd  "/content/gdrive/Shared drives/Datasets/sridhar-30-days/gan_mnist"

/content/gdrive/Shared drives/Datasets/sridhar-30-days/gan_mnist


In [ ]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
import torch.optim as Optim
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

In [ ]:
transform_option = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
image_train = MNIST(root="./data/MNIST", download=True, train=True, transform=transform_option)
train_data_loader = DataLoader(image_train, batch_size=32, shuffle=True)
image_test = MNIST(root="./data/MNIST", download=True, train=False, transform=transform_option)
test_data_loader = DataLoader(image_test, batch_size=32, shuffle=True)

In [ ]:
class MNISTDiscriminator(nn.Module):
  def __init__(self):
    super(MNISTDiscriminator, self).__init__()
    self.conv1 = nn.Conv2d(1, 24, 3)
    self.conv2 = nn.Conv2d(24, 48, 3)
    self.fc1 = nn.Linear(48, 120)
  
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = torch.sigmoid(self.fc1(x))
    return x


In [ ]:
class MNISTGenerator(nn.Module):
  def __init__(self):
    super(MNISTGenerator, self).__init__()
    self.conv1 = nn.ConvTranspose2d(100, 24, 3)
    self.conv2 = nn.ConvTranspose2d(24, 1, 3)
  
  def forward(self, x):
    x = F.relu(nn.BatchNorm2d(self.conv1(x)))
    x = F.relu(self.conv2(x))
    
    return x

In [ ]:
discriminator = MNISTDiscriminator()
generator = MNISTGenerator()

In [ ]:
gpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

In [ ]:
generator = generator.to(gpu)
discriminator = discriminator.to(gpu)

In [ ]:
generator.parameters()

<generator object Module.parameters at 0x7f34f5c59200>

In [ ]:
generator_optimizer = Optim.Adam(params=generator.parameters(), lr=0.01)
discriminator_optimizer = Optim.Adam(params=discriminator.parameters(), lr=0.01)

In [ ]:
generator.train()
discriminator.train()
generator_loss_for_epoch = []
discriminator_loss_for_epoch = []
number_of_batch = 0

for epoch in range(100):
  for image_batch, item in train_data_loader:
    print( image_batch.size(0))
    image_batch = image_batch.to(gpu)
    label_real = torch.ones(image_batch.size(0), device=gpu)
    label_fake =  torch.zeros(image_batch.size(0), device=gpu)
    noise = torch.randn(image_batch.size(0), 100,1,1, device=gpu )
    generator_image_batch = generator(noise)


    real_prediction = discriminator(image_batch).squeeze()
    fake_prediction = discriminator(generator_image_batch.detach()).squeeze()
    discriminator_loss = 0.5 * (F.binary_cross_entropy(real_prediction,label_real) + F.binary_cross_entropy(fake_prediction,label_fake))
    discriminator_optimizer.zero_grad()
    discriminator_loss.backward()
    discriminator_optimizer.step()


    fake_prediction = discriminator(generator_image_batch).squeeze()
    generator_loss = F.binary_cross_entropy(fake_prediction,label_real)
    generator_optimizer.zero_grad()
    generator_loss.backward()
    generator_optimizer.step()
    
    generator_loss_for_epoch[-1] += generator_loss.item()
    discriminator_loss_for_epoch[-1] += discriminator_loss.item()

    number_of_batch += 1
  print("Previously performed epoch training:" + epoch + "number")
  print("Previously performed epoch generator loss:" + generator_loss_for_epoch[-1] / number_of_batch)
  print("Previously performed epoch discriminator loss:" + discriminator_loss_for_epoch[-1] / number_of_batch)

